In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
import json
import dotenv

In [19]:
dotenv.load_dotenv()

True

In [20]:
from typing import TypedDict, Annotated, List, Dict, Literal
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Image
import operator
import grandalf
import pygraphviz

In [21]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [22]:
from langgraph.graph import START, END, StateGraph
from langgraph.types import Command, Send
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_experimental.utilities import PythonREPL
from langgraph.graph.message import add_messages

In [23]:
#from src.utils_email import send_report_via_email
from src.utils_trans import search_flight_details

ModuleNotFoundError: No module named 'pyflightdata'

In [ ]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )

In [ ]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [ ]:
class State(TypedDict):
    messages: Annotated[List, add_messages]
    trip_start_date: str
    trip_end_date: str
    traveler_preferences: Dict

    # Output of day_to_split_node
    split_decision: Dict  # {"should_split": bool, "segments": [{location, start_date, end_date}]}
    segments: List[Dict]  # normalized segments

    # Map branch intermediates per segment id
    segment_pools: Annotated[List[Dict], operator.add]  # [{segment_id, activities: [...] }]
    segment_schedules: Annotated[List[Dict], operator.add]  # [{segment_id, schedule: [...]}]

    # Aggregation
    total_schedule: Dict  # {days: [...], transitions: [...]} 
    report_markdown: str


### Transportation Research Agent

In [ ]:
@tool
def get_flight_data(
) -> str:
    """

    Retrieve location and time information for a round-trip flight to assist with travel planning.

    Returns:
        start_flight (str): Flight number for the departure flight.
        return_flight (str): Flight number for the return flight.
    """

    result = search_flight_details()
    return json.dumps(result)

In [ ]:
#get_flight_data()

In [ ]:
flight_data_agent = create_react_agent(
    llm,
    tools=[get_flight_data],
    prompt=get_system_prompt(
        "You are a travel planning assistant, specializing in retrieving and presenting round-trip flight information. Your primary tool, get_flight_data, accepts two flight numbers (departure and return) and returns a JSON string with details including departure and arrival airports, regions, and times."
    ),
)

### Activities (Weather focused) Agent

### Schedule Generate Agent

### Define Graph

In [ ]:
def flight_data_node(state: State) -> State:
    start_flight = state.get('start_flight') or "extract_from_messages"
    return_flight = state.get('return_flight') or "extract_from_messages"
    
    # Run agent (assuming it calls tool and gets JSON)
    agent_output = flight_data_agent.invoke({"messages": state['messages'] + [HumanMessage(content=f"Get data for {start_flight} and {return_flight}")]})
    
    # Parse and update state
    flight_info = json.loads(agent_output['output'])  # Or similar
    state['flight_info'] = flight_info
    state['start_date'] = "parse_from_flight_info"  # e.g., extract scheduled date
    state['return_date'] = "parse_from_flight_info"
    state['messages'].append(SystemMessage(content="Flight info updated."))
    return state

In [ ]:
import uuid
from datetime import datetime, timedelta
from typing import Any, Dict


def _invoke_json(system: str, user: str) -> Dict[str, Any]:
    messages = [SystemMessage(content=system), HumanMessage(content=user)]
    ai = llm.invoke(messages)
    content = ai.content
    try:
        return json.loads(content)
    except Exception:
        start = content.find("{")
        end = content.rfind("}")
        if start != -1 and end != -1 and end > start:
            return json.loads(content[start:end + 1])
        raise


def _date_range_days(start_date: str, end_date: str) -> int:
    start = datetime.fromisoformat(start_date).date()
    end = datetime.fromisoformat(end_date).date()
    return (end - start).days + 1


def day_to_split_node(state: State) -> State:
    system = (
        "You split trips into logical city segments. Output STRICT JSON only with keys: "
        "split_decision: {should_split: bool, rationale: string}, "
        "segments: [{segment_id, location, start_date, end_date, days}]. "
        "Ensure segments fully cover the trip with no gaps/overlaps and ISO dates."
    )
    user = json.dumps({
        "trip_start_date": state.get("trip_start_date") or state.get("start_date"),
        "trip_end_date": state.get("trip_end_date") or state.get("return_date"),
        "preferences": state.get("traveler_preferences", {}),
        "message": state.get("messages", [])[-1].content if state.get("messages") else "",
    })
    result = _invoke_json(system, user)

    segments = result.get("segments", [])
    normalized = []
    for seg in segments:
        seg_id = seg.get("segment_id") or str(uuid.uuid4())
        loc = seg["location"]
        s = seg["start_date"]
        e = seg["end_date"]
        days = seg.get("days") or _date_range_days(s, e)
        normalized.append({
            "segment_id": seg_id,
            "location": loc,
            "start_date": s,
            "end_date": e,
            "days": days,
        })

    state["split_decision"] = result.get("split_decision", {})
    state["segments"] = normalized
    return state


def activities_pool_node(state: State) -> State:
    seg = state["current_segment"]
    system = (
        "Generate a diverse activity pool tailored to the city and days. "
        "Return STRICT JSON: {segment_id, activities: ["
        "{name, category, est_duration_min, opening_hours, area, cost_level, notes}]}. "
        "Include food, culture, light hiking options based on preferences."
    )
    user = json.dumps({
        "segment": seg,
        "preferences": state.get("traveler_preferences", {}),
        "target_pool_size": 12,
    })
    result = _invoke_json(system, user)
    entry = {
        "segment_id": seg["segment_id"],
        "activities": result.get("activities", [])[:15],
    }
    # Preserve current_segment for the next node in this branch
    return {"segment_pools": [entry], "current_segment": seg}


def daily_schedule_node(state: State) -> State:
    seg = state["current_segment"]
    pools_by_id = {p["segment_id"]: p for p in state.get("segment_pools", [])}
    pool = pools_by_id.get(seg["segment_id"], {"activities": []})

    system = (
        "Create daily schedules from the candidate pool. Respect transit time, "
        "opening hours, and breaks. Each day ~6-9 hours of activities. "
        "Return STRICT JSON: {segment_id, schedule: ["
        "{date, items: [{start_min, end_min, type, name, notes}]}]}. "
        "Times are minutes from 00:00. Include meals and rest."
    )
    user = json.dumps({
        "segment": seg,
        "activities_pool": pool,
        "assumptions": {
            "default_open": "09:00-20:00",
            "avg_transit_min": 20,
            "lunch_window": "12:00-13:00",
            "dinner_window": "18:00-19:00",
        },
    })
    result = _invoke_json(system, user)
    entry = {
        "segment_id": seg["segment_id"],
        "schedule": result.get("schedule", []),
    }
    return {"segment_schedules": [entry]}


def total_schedule_node(state: State) -> State:
    schedules_by_id = {s["segment_id"]: s["schedule"] for s in state.get("segment_schedules", [])}
    ordered_segments = sorted(state.get("segments", []), key=lambda s: s["start_date"]) 

    days_out = []
    transitions = []
    for i, seg in enumerate(ordered_segments):
        for day in schedules_by_id.get(seg["segment_id"], []):
            days_out.append({
                "segment_id": seg["segment_id"],
                "location": seg["location"],
                "date": day.get("date"),
                "items": day.get("items", []),
            })
        if i > 0:
            prev = ordered_segments[i - 1]
            transitions.append({
                "from": prev["location"],
                "to": seg["location"],
                "date": seg["start_date"],
                "notes": "Inter-city transfer between segments",
            })

    state["total_schedule"] = {"days": days_out, "transitions": transitions}
    return state


In [ ]:
def report_node(state: State) -> State:
    total = state.get("total_schedule", {"days": [], "transitions": []})
    lines = []
    lines.append(f"# Trip Plan: {state['trip_start_date']} to {state['trip_end_date']}")
    lines.append("")
    # Overview
    lines.append("## Overview")
    lines.append("- Segments:")
    for seg in sorted(state.get("segments", []), key=lambda s: s["start_date"]):
        lines.append(f"  - {seg['location']}: {seg['start_date']} → {seg['end_date']} ({seg['days']} days)")
    if total.get("transitions"):
        lines.append("- Transitions:")
        for t in total["transitions"]:
            lines.append(f"  - {t['date']}: {t['from']} → {t['to']} — {t['notes']}")
    lines.append("")

    # Daily tables
    lines.append("## Daily Schedule")
    for day in sorted(total.get("days", []), key=lambda d: d.get("date", "")):
        lines.append(f"### {day.get('date', '')} — {day.get('location', '')}")
        lines.append("| Time | Type | Name | Notes |")
        lines.append("|------|------|------|-------|")
        for item in day.get("items", []):
            start = item.get("start_min", 0)
            end = item.get("end_min", 0)
            def _hm(m):
                h = int(m // 60)
                mm = int(m % 60)
                return f"{h:02d}:{mm:02d}"
            time_str = f"{_hm(start)}–{_hm(end)}" if end else _hm(start)
            lines.append(f"| {time_str} | {item.get('type','activity')} | {item.get('name','')} | {item.get('notes','')} |")
        lines.append("")

    state["report_markdown"] = "\n".join(lines)
    return state

In [ ]:
def split_router(state: State):
    # Fan-out per segment
    return [Send("activities_pool_agent", {"current_segment": seg}) for seg in state.get("segments", [])]

In [ ]:
def join_router(state: State):
    # Proceed when we have a schedule for every segment
    num_segs = len(state.get("segments", []))
    have = len({s.get("segment_id") for s in state.get("segment_schedules", [])})
    if num_segs > 0 and have >= num_segs:
        return "total_schedule_agent"
    return "__end__"



In [ ]:
workflow = StateGraph(State)

# Core nodes
workflow.add_node("day_to_split_agent", day_to_split_node)
workflow.add_node("activities_pool_agent", activities_pool_node)
workflow.add_node("daily_schedule_agent", daily_schedule_node)
workflow.add_node("join_agent", join_func)
workflow.add_node("total_schedule_agent", total_schedule_node)
workflow.add_node("report_agent", report_node)

# Fan-out per segment and gather back
workflow.add_conditional_edges("day_to_split_agent", split_router)
workflow.add_edge("activities_pool_agent", "daily_schedule_agent")
workflow.add_edge("daily_schedule_agent", "join_agent")
workflow.add_conditional_edges("join_agent", join_router)
workflow.add_edge("total_schedule_agent", "report_agent")
workflow.add_edge("report_agent", END)

# Entry point
workflow.set_entry_point("day_to_split_agent")
graph = workflow.compile()

In [9]:
# Try ASCII
try:
    graph.get_graph().print_ascii()
except ImportError as e:
    print("ASCII drawing requires grandalf. Install with: pip install grandalf\n", e)

# Try PNG
try:
    output_path = "graph.png"
    graph.get_graph().draw_png(output_path)
    print(f"PNG graph written to {output_path}")
except ImportError as e:
    print("PNG drawing requires pygraphviz. Install with: pip install pygraphviz\n", e)

NameError: name 'graph' is not defined

### Run Graph

In [25]:
input_message = HumanMessage(
    content=(
        "Plan a 10-day Japan trip arriving Kansai (Osaka). Focus on food, culture, "
        "light hiking, and efficient transit."
    )
)

In [26]:
TRIP_START = "2025-03-10"
TRIP_END = "2025-03-19"
PREFERENCES = {
    "pace": "moderate",
    "interests": ["food", "culture", "light hiking"],
    "budget": "mid",
}

#EMAIL_ADDRESS = "terence2379@gmail.com"

In [ ]:
events = graph.stream(
    input={
        "messages": [input_message],
        "trip_start_date": TRIP_START,
        "trip_end_date": TRIP_END,
        "traveler_preferences": PREFERENCES,
    },
    config={"recursion_limit": 30},
    stream_mode="values",
)
final_report = None
for event in events:
    if "report_markdown" in event:
        final_report = event["report_markdown"]

if final_report:
    display(Markdown(final_report))
else:
    print("No final report produced. Check inputs or graph configuration.")

KeyError: 'current_segment'

### Send Email


In [ ]:
#body = message.content

#title = llm.invoke(
#    f"Extract the subject from this report content as the email subject and return the title directly without any introductory text: {body}")

#subject = title.content

#result = send_report_via_email(subject, body, EMAIL_ADDRESS)
#print(result)